Adjusted close stock price for Apple, Walmart and Tesla.

In [1]:
import pandas as pd

In [2]:
df_apple = pd.read_csv('C:/Users/Mohan/Documents/Machine Learning R_27.07.21/Machine Learning Projects 141 - Granger Causality Test/AAPL.csv')
df_walmart = pd.read_csv('C:/Users/Mohan/Documents/Machine Learning R_27.07.21/Machine Learning Projects 141 - Granger Causality Test/WMT.csv')
df_tesla = pd.read_csv('C:/Users/Mohan/Documents/Machine Learning R_27.07.21/Machine Learning Projects 141 - Granger Causality Test/TSLA.csv')

In [3]:
df_apple.shape, df_walmart.shape, df_tesla.shape

((2638, 7), (2638, 7), (2638, 7))

In [4]:
df_tesla.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
2633,2020-12-14,619.000000,642.750000,610.200012,639.830017,639.830017,52040600
2634,2020-12-15,643.280029,646.900024,623.799988,633.250000,633.250000,45223600
2635,2020-12-16,628.229980,632.500000,605.000000,622.770020,622.770020,42095800
2636,2020-12-17,628.190002,658.820007,619.500000,655.900024,655.900024,56270100
2637,2020-12-18,668.900024,695.000000,628.539978,695.000000,695.000000,222126200


In [5]:
df = pd.merge(df_apple[['Date', 'Adj Close']], df_walmart[['Date', 'Adj Close']], on='Date', how='right').rename(columns = {'Adj Close_x':'apple', 'Adj Close_y':'walmart'})
df = df.merge(df_tesla[['Date', 'Adj Close']], on='Date', how='right').rename(columns={'Adj Close':'tesla'})

In [6]:
df_apple.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-06-30,9.168214,9.213214,8.928928,8.983214,7.749377,739452000
1,2010-07-01,9.082143,9.100000,8.686429,8.874286,7.655411,1022896000
2,2010-07-02,8.946072,8.961785,8.685715,8.819285,7.607966,693842800
3,2010-07-06,8.964286,9.028571,8.791429,8.879642,7.660032,615235600
4,2010-07-07,8.946072,9.241786,8.919642,9.238214,7.969353,654556000


In [7]:
df.head()

,Date,apple,walmart,tesla
0,2010-06-30,7.749377,37.116917,4.766
1,2010-07-01,7.655411,37.325382,4.392
2,2010-07-02,7.607966,37.062855,3.840
3,2010-07-06,7.660032,37.502975,3.222
4,2010-07-07,7.969353,37.773232,3.160


In [8]:
df['Date'] =  pd.to_datetime(df['Date'])
# df = df.set_index('Date')

In [9]:
df = df.set_index('Date').rename_axis('company', axis=1)

In [10]:
df.head()

company,apple,walmart,tesla
Date,,,
2010-06-30,7.749377,37.116917,4.766
2010-07-01,7.655411,37.325382,4.392
2010-07-02,7.607966,37.062855,3.840
2010-07-06,7.660032,37.502975,3.222
2010-07-07,7.969353,37.773232,3.160


In [11]:
df.columns

Index(['apple', 'walmart', 'tesla'], dtype='object', name='company')

In [12]:
df.isnull().sum()

company
apple      0
walmart    0
tesla      0
dtype: int64

## Visualize the Time Series

In [13]:
import plotly.express as px

fig = px.line(df, facet_col="company", facet_col_wrap=1)
fig.update_yaxes(matches=None)
fig.show()

c:\Users\Mohan\anaconda3\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  v = v.dt.to_pydatetime()


In [14]:
fig = px.area(df, facet_col='company', facet_col_wrap=1)
fig.update_yaxes(matches=None)
fig.show()

c:\Users\Mohan\anaconda3\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



Time series data preparation process has several steps we consider as data mining process.

Cross-check ADF test and KPSS test.

## ADF Test for Stationarity

Null hypothesis: If failed to be rejected, it suggests the time series is not stationarity

Alternative hypothesis: The null hypothesis is rejected, it suggests the time series is stationary

In [15]:
n_obs = 20
df_train, df_test = df[0:-n_obs], df[-n_obs:]

In [16]:
df_train.shape, df_test.shape

((2618, 3), (20, 3))

In [17]:
from statsmodels.tsa.stattools import adfuller

def adf_test(df):
    result = adfuller(df.values)
    print('ADF Statistics: %f' % result[0])
    print('p-value: %f' % result[1])
    print('Critical values:')
    for key, value in result[4].items():
        print('\t%s: %.3f' % (key, value))

In [18]:
print('ADF Test: Apple time series')
adf_test(df_train['apple'])
print('ADF Test: Walmart time series')
adf_test(df_train['walmart'])
print('ADF Test: Tesla time series')
adf_test(df_train['tesla'])

ADF Test: Apple time series
ADF Statistics: 2.956336
p-value: 1.000000
Critical values:
	1%: -3.433
	5%: -2.863
	10%: -2.567
ADF Test: Walmart time series
ADF Statistics: 1.836596
p-value: 0.998419
Critical values:
	1%: -3.433
	5%: -2.863
	10%: -2.567
ADF Test: Tesla time series
ADF Statistics: 5.696340
p-value: 1.000000
Critical values:
	1%: -3.433
	5%: -2.863
	10%: -2.567


The p-values are all well above the 0.05 alpha level, we cannot reject the null hypothesis. So the three time series are NOT stationary.

## KPSS test for Stationary

Null hypothesis: The time series is stationarity

Alternative hypothesis: The time series is non-stationary

In [19]:
from statsmodels.tsa.stattools import kpss

def kpss_test(df):    
    statistic, p_value, n_lags, critical_values = kpss(df.values)
    
    print(f'KPSS Statistic: {statistic}')
    print(f'p-value: {p_value}')
    print(f'num lags: {n_lags}')
    print('Critial Values:')
    for key, value in critical_values.items():
        print(f'   {key} : {value}')

In [20]:
print('KPSS Test: Apple time series')
kpss_test(df_train['apple'])
print('KPSS Test: Walmart time series')
kpss_test(df_train['walmart'])
print('KPSS Test: Tesla time series')
kpss_test(df_train['tesla'])

KPSS Test: Apple time series
KPSS Statistic: 6.239496496540852
p-value: 0.01
num lags: 30
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
KPSS Test: Walmart time series
KPSS Statistic: 6.745312462682414
p-value: 0.01
num lags: 30
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
KPSS Test: Tesla time series
KPSS Statistic: 3.4921824148140144
p-value: 0.01
num lags: 30
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739


C:\Users\Mohan\AppData\Local\Temp\ipykernel_4180\2516723409.py:4: InterpolationWarning:

The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.


C:\Users\Mohan\AppData\Local\Temp\ipykernel_4180\2516723409.py:4: InterpolationWarning:

The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.


C:\Users\Mohan\AppData\Local\Temp\ipykernel_4180\2516723409.py:4: InterpolationWarning:

The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.




The p-value are all less than 0.05 alpha level, we can reject the null hypothesis and derive that the three time series are NOT stationary.

After cross-check ADF test and KPSS test. We can conclude that the three time series data we have here are not stationary. We will transform the time series to be stationary by difference method.

In [21]:
df_train_transformed = df_train.diff().dropna()

In [22]:
fig = px.line(df_train_transformed, facet_col="company", facet_col_wrap=1)
fig.update_yaxes(matches=None)
fig.show()

c:\Users\Mohan\anaconda3\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [23]:
print('ADF Test: Apple time series transformed')
adf_test(df_train_transformed['apple'])
print('ADF Test: Walmart time series transformed')
adf_test(df_train_transformed['walmart'])
print('ADF Test: Tesla time series transformed')
adf_test(df_train_transformed['tesla'])

ADF Test: Apple time series transformed
ADF Statistics: -8.943347
p-value: 0.000000
Critical values:
	1%: -3.433
	5%: -2.863
	10%: -2.567
ADF Test: Walmart time series transformed
ADF Statistics: -10.706616
p-value: 0.000000
Critical values:
	1%: -3.433
	5%: -2.863
	10%: -2.567
ADF Test: Tesla time series transformed
ADF Statistics: -8.327248
p-value: 0.000000
Critical values:
	1%: -3.433
	5%: -2.863
	10%: -2.567


After transform the data, the p-values are all well below the 0.05 alpha level, we reject the null hypothesis. So the current data is stationary.

In [24]:
print('KPSS Test: Apple time series transformed')
kpss_test(df_train_transformed['apple'])
print('KPSS Test: Walmart time series transformed')
kpss_test(df_train_transformed['walmart'])
print('KPSS Test: Tesla time series transformed')
kpss_test(df_train_transformed['tesla'])

KPSS Test: Apple time series transformed
KPSS Statistic: 0.7125474666259631
p-value: 0.012404775761276078
num lags: 7
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
KPSS Test: Walmart time series transformed
KPSS Statistic: 0.37527059987233735
p-value: 0.08781439660675114
num lags: 15
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
KPSS Test: Tesla time series transformed
KPSS Statistic: 1.0872104550886064
p-value: 0.01
num lags: 23
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739


C:\Users\Mohan\AppData\Local\Temp\ipykernel_4180\2516723409.py:4: InterpolationWarning:

The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.




Some of the KPSS Null Hypothesis could not be rejected.

## VAR model

The VAR class assumes that the passed time series are stationary. Non-stationary or trending data can often be transformed to be stationary by first-differencing or some other method.

Decide the Order (P) of VAR model : For our data decide on which lag is better to fit model. Choosed lowest AIC and other metrics.

In [25]:
from statsmodels.tsa.api import VAR

model = VAR(df_train_transformed)
for i in [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]:
    result = model.fit(i)
    print('Lag Order =', i)
    print('AIC : ', result.aic)
    print('BIC : ', result.bic)
    print('FPE : ', result.fpe)
    print('HQIC: ', result.hqic, '\n')

c:\Users\Mohan\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.



Lag Order = 1
AIC :  2.4466464459186046
BIC :  2.4735703069854718
FPE :  11.549549744739021
HQIC:  2.4563985747100356 

Lag Order = 2
AIC :  2.437260328541744
BIC :  2.484392032903686
FPE :  11.441651837345752
HQIC:  2.4543322998538724 

Lag Order = 3
AIC :  2.424830511032504
BIC :  2.492182885487563
FPE :  11.300315243753573
HQIC:  2.4492272557453414 

Lag Order = 4
AIC :  2.4193708456609766
BIC :  2.5069567302209856
FPE :  11.23878892699503
HQIC:  2.451097299882335 

Lag Order = 5
AIC :  2.3707142484088135
BIC :  2.478546496318087
FPE :  10.705040529990338
HQIC:  2.4097753534818374 

Lag Order = 6
AIC :  2.362436591580944
BIC :  2.490528069335004
FPE :  10.616796922589288
HQIC:  2.408837294091657 

Lag Order = 7
AIC :  2.349536903526385
BIC :  2.4979004908907165
FPE :  10.480727572667528
HQIC:  2.4032821553112482 

Lag Order = 8
AIC :  2.310903005930652
BIC :  2.4795515959594963
FPE :  10.083543773434485
HQIC:  2.3719977640841376 

Lag Order = 9
AIC :  2.287000947587056
BIC :  2.4759

There is no hard-and-fast-rule on the choice of lag order. It is basically an empirical issue. However, it is often advised to use the AIC in selecting the lag order with the smallest value. Here we will select lag order = 15.

In [26]:
model.select_order(15)

Select the lowest AIC lag.

In [27]:
results = model.fit(maxlags=15, ic='aic')
results.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Wed, 27, Dec, 2023
Time:                     16:56:30
--------------------------------------------------------------------
No. of Equations:         3.00000    BIC:                    2.48026
Nobs:                     2602.00    HQIC:                   2.28193
Log likelihood:          -13760.4    FPE:                    8.75181
AIC:                      2.16925    Det(Omega_mle):         8.30359
--------------------------------------------------------------------
Results for equation apple
                 coefficient       std. error           t-stat            prob
------------------------------------------------------------------------------
const               0.051964         0.016711            3.110           0.002
L1.apple           -0.113389         0.022943           -4.942           0.000
L1.walmart         -0.075488         0.017381           -4.343      

The biggest correlation is 0.43 (Apple & Tesla).

## Check the Durbin-Watson statistic

Serial correlation of residuals is used to check if there is any leftover pattern in the residuals (errors). If there is any correlation left in the residuals, then, there is some pattern in the time series that is still left to be explained by the model. In that case, the typical course of action is to either increase the order of the model or induce more predictors into the system or look for a different algorithm to model the time series.

In [28]:
from statsmodels.stats.stattools import durbin_watson

out = durbin_watson(results.resid)

for col, val in zip(df.columns, out):
    print(col, ':', round(val, 2))

apple : 2.0
walmart : 2.0
tesla : 2.0


A value of 2.0 means that there is no autocorrelation detected in the residuals.

## Granger Causality Test

In [29]:
import numpy as np

In [30]:
from statsmodels.tsa.stattools import grangercausalitytests

maxlag=15
test = 'ssr_chi2test'

def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
   
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

grangers_causation_matrix(df_train_transformed, variables = df_train_transformed.columns)

c:\Users\Mohan\anaconda3\Lib\site-packages\statsmodels\tsa\stattools.py:1488: FutureWarning:

verbose is deprecated since functions should not print results

c:\Users\Mohan\anaconda3\Lib\site-packages\statsmodels\tsa\stattools.py:1488: FutureWarning:

verbose is deprecated since functions should not print results

c:\Users\Mohan\anaconda3\Lib\site-packages\statsmodels\tsa\stattools.py:1488: FutureWarning:

verbose is deprecated since functions should not print results

c:\Users\Mohan\anaconda3\Lib\site-packages\statsmodels\tsa\stattools.py:1488: FutureWarning:

verbose is deprecated since functions should not print results

c:\Users\Mohan\anaconda3\Lib\site-packages\statsmodels\tsa\stattools.py:1488: FutureWarning:

verbose is deprecated since functions should not print results

c:\Users\Mohan\anaconda3\Lib\site-packages\statsmodels\tsa\stattools.py:1488: FutureWarning:

verbose is deprecated since functions should not print results

c:\Users\Mohan\anaconda3\Lib\site-packages\statsmode

,apple_x,walmart_x,tesla_x
apple_y,1.0,0.0,0.0
walmart_y,0.0,1.0,0.0
tesla_y,0.0,0.0,1.0


The row are the Response (Y) and the columns are the predictor series (X). If a given p-value is < significance level (0.05), for example, when we take the value 0.0 in (row 1, column 2), we can reject the null hypothesis and conclude that walmart_x Granger causing apple_y. Likewise, the 0.0 in (row 2, column 1) refers to walmart_y Granger causing apple_x.

All the time series in the above data are interchangeably Granger causing each other.

In [31]:
lag_order = results.k_ar
print(lag_order)

15


## Forecast

In [32]:
df_input = df_train_transformed.values[-lag_order:]
df_forecast = results.forecast(y=df_input, steps=n_obs)
df_forecast = (pd.DataFrame(df_forecast, index=df_test.index, columns=df_test.columns + '_pred'))

In [33]:
def invert_transformation(df, pred):
    forecast = df_forecast.copy()
    columns = df.columns
    for col in columns:
        forecast[str(col)+'_pred'] = df[col].iloc[-1] + forecast[str(col)+'_pred'].cumsum()
    return forecast
output = invert_transformation(df_train, df_forecast)

In [34]:
output

company,apple_pred,walmart_pred,tesla_pred
Date,,,
2020-11-20,119.526205,151.759086,509.314704
2020-11-23,120.218499,153.136059,516.392665
2020-11-24,119.914257,153.234075,510.919619
2020-11-25,118.250224,152.394891,498.043869
2020-11-27,117.696289,152.065309,494.527580
2020-11-30,118.152242,152.838528,489.821496
2020-12-01,118.581646,153.648201,485.367727
2020-12-02,118.872377,153.622405,486.124842
2020-12-03,120.841659,154.285421,497.479445


In [35]:
combined = pd.concat([output['apple_pred'], df_test['apple'], output['walmart_pred'], df_test['walmart'], output['tesla_pred'], df_test['tesla']], axis=1)

In [36]:
combined

,apple_pred,apple,walmart_pred,walmart,tesla_pred,tesla
Date,,,,,,
2020-11-20,119.526205,117.339996,151.759086,149.692825,509.314704,489.609985
2020-11-23,120.218499,113.849998,153.136059,150.380295,516.392665,521.849976
2020-11-24,119.914257,115.169998,153.234075,150.808746,510.919619,555.380005
2020-11-25,118.250224,116.029999,152.394891,151.277039,498.043869,574.000000
2020-11-27,117.696289,116.589996,152.065309,151.047882,494.527580,585.760010
2020-11-30,118.152242,119.050003,152.838528,152.233536,489.821496,567.599976
2020-12-01,118.581646,122.720001,153.648201,152.084076,485.367727,584.760010
2020-12-02,118.872377,123.080002,153.622405,149.971802,486.124842,568.820007
2020-12-03,120.841659,122.940002,154.285421,148.756256,497.479445,593.380005


In [37]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


rmse = mean_squared_error(combined['apple_pred'], combined['apple'], squared=False)
mae = mean_absolute_error(combined['apple_pred'], combined['apple'])

print('Forecast accuracy of Apple')
print('RMSE: ', round(rmse,2))
print('MAE: ', round(mae,2))

Forecast accuracy of Apple
RMSE:  7.31
MAE:  6.0


In [38]:
rmse = mean_squared_error(combined['walmart_pred'], combined['walmart'], squared=False)
mae = mean_absolute_error(combined['walmart_pred'], combined['walmart'])

print('Forecast accuracy of Walmart')
print('RMSE: ', round(rmse,2))
print('MAE: ', round(mae,2))

Forecast accuracy of Walmart
RMSE:  5.05
MAE:  4.52


In [39]:
rmse = mean_squared_error(combined['tesla_pred'], combined['tesla'], squared=False)
mae = mean_absolute_error(combined['tesla_pred'], combined['tesla'])

print('Forecast accuracy of Tesla')
print('RMSE: ', round(rmse,2))
print('MAE: ', round(mae,2))

Forecast accuracy of Tesla
RMSE:  125.16
MAE:  113.18
